<a href="https://colab.research.google.com/github/violafjrnnn/Coursera1/blob/main/dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir ~/.kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"violafajrin","key":"b40aaaf4aef9c0a827ede12dece1a402"}'}

In [3]:
#mengubah akses file kaggle kredensial
!mv kaggle.json ~/.kaggle/

In [4]:
#import dataset ke dalam folder konten
!kaggle datasets download -d niten19/face-shape-dataset

 97% 617M/639M [00:05<00:00, 116MB/s]
100% 639M/639M [00:05<00:00, 124MB/s]


In [5]:
#mengeskstrak file dataset

import zipfile
zip_file = zipfile.ZipFile('/content/face-shape-dataset.zip')
zip_file.extractall('/tmp/')

In [6]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [7]:
#mengimport ImageDataGenerator

from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import glob
import h5py
from PIL import Image
from keras.models import Sequential, Model
from keras.applications.vgg16 import VGG16, preprocess_input
#from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, SeparableConv2D
from keras.layers import GlobalMaxPooling2D
#from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import to_categorical
from keras import backend as K
import numpy as np

In [8]:
batch_size = 16  #was 16
train_images = '../tmp/FaceShape Dataset/training_set'
test_images = '../tmp/FaceShape Dataset/testing_set'

TrainDatagen = ImageDataGenerator(
        preprocessing_function= preprocess_input,
        horizontal_flip = True,
        vertical_flip=True
)

TestDatagen = ImageDataGenerator(
    preprocessing_function= preprocess_input

)

train_data = TrainDatagen.flow_from_directory(
    train_images,
    target_size = (224,224),
    batch_size =batch_size,
    class_mode = 'categorical'


)

test_data = TestDatagen.flow_from_directory(
    test_images,
    target_size = (224,224),
    batch_size = batch_size,
    class_mode = 'categorical'
)

Found 4000 images belonging to 5 classes.
Found 1000 images belonging to 5 classes.


In [9]:
import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization

def build_model():
    input_img = Input(shape=(224,224,3), name='ImageInput')
    x = Conv2D(64, (3,3), activation='relu', padding='same', name='Conv1_1')(input_img)
    x = Conv2D(64, (3,3), activation='relu', padding='same', name='Conv1_2')(x)
    x = MaxPooling2D((2,2), name='pool1')(x)
    
    x = SeparableConv2D(128, (3,3), activation='relu', padding='same', name='Conv2_1')(x)
    x = SeparableConv2D(128, (3,3), activation='relu', padding='same', name='Conv2_2')(x)
    x = MaxPooling2D((2,2), name='pool2')(x)
    
    x = SeparableConv2D(256, (3,3), activation='relu', padding='same', name='Conv3_1')(x)
    x = BatchNormalization(name='bn1')(x)
    x = SeparableConv2D(256, (3,3), activation='relu', padding='same', name='Conv3_2')(x)
    x = BatchNormalization(name='bn2')(x)
    x = SeparableConv2D(256, (3,3), activation='relu', padding='same', name='Conv3_3')(x)
    x = MaxPooling2D((2,2), name='pool3')(x)
    
    x = SeparableConv2D(512, (3,3), activation='relu', padding='same', name='Conv4_1')(x)
    x = BatchNormalization(name='bn3')(x)
    x = SeparableConv2D(512, (3,3), activation='relu', padding='same', name='Conv4_2')(x)
    x = BatchNormalization(name='bn4')(x)
    x = SeparableConv2D(512, (3,3), activation='relu', padding='same', name='Conv4_3')(x)
    x = MaxPooling2D((2,2), name='pool4')(x)
    
    x = Flatten(name='flatten')(x)
    x = Dense(1024, activation='relu', name='fc1')(x)
    x = Dropout(0.7, name='dropout1')(x)
    x = Dense(512, activation='relu', name='fc2')(x)
    x = Dropout(0.5, name='dropout2')(x)
    x = Dense(5, activation='softmax', name='fc3')(x)

    model = Model(inputs=input_img, outputs=x)
    return model

In [10]:
model =  build_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 ImageInput (InputLayer)     [(None, 224, 224, 3)]     0         
                                                                 
 Conv1_1 (Conv2D)            (None, 224, 224, 64)      1792      
                                                                 
 Conv1_2 (Conv2D)            (None, 224, 224, 64)      36928     
                                                                 
 pool1 (MaxPooling2D)        (None, 112, 112, 64)      0         
                                                                 
 Conv2_1 (SeparableConv2D)   (None, 112, 112, 128)     8896      
                                                                 
 Conv2_2 (SeparableConv2D)   (None, 112, 112, 128)     17664     
                                                                 
 pool2 (MaxPooling2D)        (None, 56, 56, 128)       0     

In [14]:
# opt = RMSprop(lr=0.0001, decay=1e-6)
opt = Adam(lr=0.0001, decay=1e-5)
es = EarlyStopping(patience=5,min_delta= .05, monitor="val_accuracy")
chkpt = ModelCheckpoint(filepath='best_model_todate', save_best_only=True, save_weights_only=True)
model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer=opt)

In [20]:
history = model.fit_generator(
    train_data,              
    steps_per_epoch = train_data.samples//batch_size,
    validation_data = test_data,
    validation_steps = test_data.samples//batch_size,
    epochs = 50,
)
model.save("test.h5")

<ipython-input-20-5c4e98390a8d>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/50
250/250 [==============================] - 84s 336ms/step - loss: 0.8446 - accuracy: 0.6743 - val_loss: 1.2571 - val_accuracy: 0.5020
Epoch 2/50
250/250 [==============================] - 82s 328ms/step - loss: 0.8036 - accuracy: 0.6902 - val_loss: 1.1972 - val_accuracy: 0.5202
Epoch 3/50
250/250 [==============================] - 82s 329ms/step - loss: 0.7363 - accuracy: 0.7150 - val_loss: 1.3732 - val_accuracy: 0.4899
Epoch 4/50
250/250 [==============================] - 81s 322ms/step - loss: 0.7185 - accuracy: 0.7245 - val_loss: 1.2810 - val_accuracy: 0.5242
Epoch 5/50
250/250 [==============================] - 73s 292ms/step - loss: 0.6858 - accuracy: 0.7398 - val_loss: 1.3447 - val_accuracy: 0.5081
Epoch 6/50
250/250 [==============================] - 82s 328ms/step - loss: 0.6439 - accuracy: 0.7602 - val_loss: 1.2784 - val_accuracy: 0.5151
Epoch 7/50
250/250 [==============================] - 81s 323ms/step - loss: 0.6040 - accuracy: 0.7690 - val_loss: 1.3187 - val_ac

In [29]:
import tensorflow as tf
!pip install tensorflowjs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [30]:
!pip install pyyaml h5py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [34]:
model.save('test.h5')

In [36]:
!mkdir ~/.model
!tensorflowjs_converter --input_format keras test.h5 model/

2023-06-05 08:58:31.491977: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [37]:
!zip -r tsjf-test-1.zip model/

  adding: model/ (stored 0%)
  adding: model/group1-shard83of100.bin (deflated 8%)
  adding: model/group1-shard44of100.bin (deflated 8%)
  adding: model/group1-shard92of100.bin (deflated 8%)
  adding: model/group1-shard85of100.bin (deflated 8%)
  adding: model/group1-shard22of100.bin (deflated 8%)
  adding: model/group1-shard53of100.bin (deflated 7%)
  adding: model/group1-shard18of100.bin (deflated 8%)
  adding: model/group1-shard20of100.bin (deflated 8%)
  adding: model/group1-shard1of100.bin (deflated 8%)
  adding: model/group1-shard30of100.bin (deflated 8%)
  adding: model/group1-shard97of100.bin (deflated 8%)
  adding: model/group1-shard65of100.bin (deflated 8%)
  adding: model/group1-shard27of100.bin (deflated 8%)
  adding: model/group1-shard10of100.bin (deflated 8%)
  adding: model/group1-shard77of100.bin (deflated 8%)
  adding: model/group1-shard96of100.bin (deflated 8%)
  adding: model/group1-shard67of100.bin (deflated 7%)
  adding: model/group1-shard45of100.bin (deflated 8%)
